In [1]:
import os
import re
import json

from tqdm import tqdm

In [2]:
from langchain_community.llms import Ollama
# llm =  Ollama(model="deepseek-r1:32b", temperature=0)

## 加载本地的GDP数据集

In [ ]:
def load_json(file, encoding="utf-8"):
    with open(file, "r", encoding=encoding) as f:
        return json.load(f)

def save_to_json(obj, file, encoding="utf-8"):
    parent_fold = os.path.dirname(file)
    os.makedirs(parent_fold, exist_ok=True)
    with open(file, "w", encoding=encoding) as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

In [4]:
dataset = load_json("../data/长沙_gdp.json")

In [5]:
PROMPT_TEMPLATE = """请你阅读长沙市的政府工作报告文本节选，抽取其中GDP具体的值。
首先会在前面的文中提到全年（表述可能为：全年、今年、去年）的国民生产总值的具体数据，然后会在后面的文本展望未来预期的国民生产总值。
如果文中出现了多个对GDP值的表述请你输出前面的。不要返回政府对未来预期的GDP的值。

文本节选如下：
{{text}}

返回json格式:
```json
{
    "reason":"说明理由",
    "GDP": "GDP值"
}
```
GDP值为数字，不需要单位，若文中未提到GDP值，请填写"<not found>"
"""

In [8]:
# # Please install OpenAI SDK first: `pip3 install openai`

# from openai import OpenAI

# client = OpenAI(base_url="http://localhost:11434/v1")

# response = client.chat.completions.create(
#     model="deepseek-r1:32b",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant"},
#         {"role": "user", "content": "Who are you?"},
#     ],
#     stream=False
# )

## 解析大模型抽取结果

In [6]:
dataset['长沙市2003.txt']

{'text': [], 'label': '<not found>', 'year': '2003', 'city': '长沙'}

In [7]:
def llm_pred(llm, dataset, output_file):
    pred_dataset = []
    for item in tqdm(dataset.values()):
        if item["text"] == []:
            continue
        text = "\n".join(item["text"])
        prompt = PROMPT_TEMPLATE.replace("{{text}}", text)
        item["prompt"] = prompt
        llm_answer = llm.invoke(prompt)
        item["llm_answer"] = llm_answer
        llm_response_text = re.search(
            r"```json\n(.*)\n```", llm_answer, re.DOTALL
        ).group(1)
        reponse_d = json.loads(llm_response_text)
        pred_label = reponse_d.get("GDP", "error")
        item["pred_label"] = pred_label
        pred_dataset.append(item)
    save_to_json(pred_dataset, output_file)

In [ ]:
qwen_7b =  Ollama(model="qwen2.5:7b", temperature=0)
llm_pred(qwen_7b, dataset, "output/长沙_gdp_qwen_7b.json")

100%|██████████| 21/21 [00:59<00:00,  2.81s/it]


In [17]:
deepseek_r1_32b =  Ollama(model="deepseek-r1:32b", temperature=0)
llm_pred(deepseek_r1_32b, dataset, "output/长沙_gdp_deepseek_r1_32b.json")

100%|██████████| 21/21 [04:45<00:00, 13.59s/it]


In [15]:
deepseek_r1_7b = Ollama(model="deepseek-r1:7b", temperature=0)
llm_pred(deepseek_r1_7b, dataset, "output/长沙_gdp_deepseek_r1_7b.json")

100%|██████████| 21/21 [01:43<00:00,  4.92s/it]


In [9]:
qwq_32b = Ollama(model="qwq:32b", temperature=0)
llm_pred(qwq_32b, dataset, "output/长沙_gdp_qwq_32b.json")

100%|██████████| 21/21 [08:34<00:00, 24.50s/it]
